#### train the original training data (new_10) with, embeddings, tfidf, and 10 created features
#### train this data after undersample, oversample for Randomforest classifier, Logistic regression
#### test on the gold dataset with those model (rfc_us, rfc_os, lr_us, lr_os)
#### rfc_us gave little better than base model
- trying the new tfidf best features (6300), random under sampling
- trying the new wcbtfidf best features (1500) random under sampling

In [18]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import joblib
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# TF IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split # fnding max_features
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
# import joblib
import os


from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV # 

## try with other models
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

import spacy
import ast

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [224]:
train_df = pd.read_csv('created_new_train_data/new_11.csv') # checkworthy_english_train dataset with new created features
# dev_df = pd.read_csv('data/CT24_checkworthy_english/CT24_checkworthy_english/CT24_checkworthy_english_dev.tsv', delimiter = '\t') # validation during training
# dev_test_df = pd.read_csv('data/CT24_checkworthy_english/CT24_checkworthy_english/CT24_checkworthy_english_dev-test.tsv', delimiter = '\t') # validation for fine tuning
test_df = pd.read_csv('created_new_test_data/english_test_11_2.csv')

In [2]:
dev_train = pd.read_csv('created_new_dev_data/first_dev_train.csv')

In [138]:
print(test_df.shape)
# print(dev_df.shape)
print(train_df.shape)

(341, 23)
(22501, 40)


In [139]:
dev_train.columns

Index(['Sentence_id', 'Text', 'class_label', 'Tokens', 'text_length',
       'cleaned_text', 'cleaned_text_length', 'roberta_sentiment',
       'roberta_sent_pos', 'roberta_sent_neg', 'roberta_sent_mixed',
       'roberta_sentiment_class', 'count_tokens', 'counts', 'names',
       'organizations', 'dates', 'punctuation_count', 'embedding',
       'converted_embedding', 'flattened_embedding', 'label', 'joined_tokens',
       'text-tfidf'],
      dtype='object')

In [225]:
train_df.drop('Unnamed: 0', axis = 1, inplace = True)
test_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [6]:
# new_df.head()

In [7]:
# new_df['converted_embedding'].iloc[0]

In [14]:
# new_df.rename(columns={'converted_embedding':'embedding'}, inplace = True)

In [8]:
# new_df.columns

In [9]:
# new_df = new_df.astype({'text_length': 'int32', 'roberta_sent_neg': 'float32', 'roberta_sent_mixed': 'float32', 'roberta_sent_pos': 'float32',
#                'names': 'int32', 'organizations': 'int32', 'dates': 'int32', 'count_tokens': 'int32', 'cleaned_text_length': 'int32', 'punctuation_count': 'int32'})

In [226]:
# ValueError: np.nan is an invalid document, expected byte or unicode string.
# drop the columns with nan value in text-idf column
train_df = train_df.dropna()

In [227]:
train_df.shape

(22385, 39)

In [228]:
# convert the embeddnigs into numpy array and then flatten the array and store

def convert_to_numpy_array(embd_str):
    #print(f"string: {embd_str}")
    #print(type(embd_str))
    embd_str_clean = embd_str.replace("\n", " ").replace(" ", " ")
    #print(type(embd_str_clean))
    #print(f"string cleansing: {embd_str_clean}")
    embd_str_clean = embd_str_clean.strip('[]')
    #print(type(embd_str_clean))
    #print(f"removing the outer brackets {embd_str_clean}")
    embd_list = embd_str_clean.split()
    #print(f"Split the string into individual values {embd_list}")
    embed_array = np.array(embd_list, dtype = float)
    #print(f"list to array {embed_array}")
    return embed_array

In [229]:
train_df['converted_embedding'] = train_df['converted_embedding'].apply(convert_to_numpy_array)

In [232]:
pca_path = 'models/pca_356.pkl'
pca = joblib.load(pca_path)

In [233]:
# pca = PCA(n_components = 356)
reduced_embeddings = pca.fit_transform(train_df['converted_embedding'].to_list())
# reduced_embeddings.shape

In [162]:
save_folder = 'models'
os.makedirs(save_folder, exist_ok = True)
model_path = os.path.join(save_folder, 'pca_356.pkl')
joblib.dump(pca, model_path)

['models\\pca_356.pkl']

In [234]:
text = train_df['text-tfidf']
vectorizer_path = 'models/tfidf_vectorizer_4300.pkl'
tfidf_vect = joblib.load(vectorizer_path) ## loading the trained tfidf vectorizer
tfidf_features = tfidf_vect.transform(text)

In [235]:
additional_features = train_df[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

In [113]:
# new_df['converted_embedding'] = new_df['embedding'].apply(convert_to_numpy_array)
# pca = PCA(n_components = 50)
# reduced_embeddings = pca.fit_transform(new_df['converted_embedding'].to_list())

In [236]:
embeddings_np = np.array(reduced_embeddings)
tfidf_features_np = tfidf_features.toarray()
additional_features_np = np.array(additional_features)

print(embeddings_np.shape)
print(tfidf_features_np.shape)
print(additional_features_np.shape)

(22385, 356)
(22385, 4300)
(22385, 10)


In [237]:
X_train = np.hstack([tfidf_features_np,embeddings_np, additional_features_np])
y_train = train_df['labels']

In [150]:
print(X_train.shape)
print(y_train.shape)

(22385, 4666)
(22385,)


In [48]:
# X_train, X_test, y_train,y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [42]:
# over sampling - MEMORY ERROR
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(random_state=42)
# X_train_balanced_o, y_train_balanced_o = smote.fit_resample(X, y)

In [238]:
# under sampling
from imblearn.under_sampling import  RandomUnderSampler
rus = RandomUnderSampler(random_state = 42)
X_train_u_balanced, y_train_u_balanced = rus.fit_resample(X_train, y_train)

In [45]:
# train RFC with balanced_data - over sampling
# clf = RandomForestClassifier()
# clf.fit(X_train_u_balanced,y_train_u_balanced)

In [53]:
# save random forest model over sampling
# save_folder = 'models'
# os.makedirs(save_folder, exist_ok = True)
# model_path = os.path.join(save_folder, 'random_forest_tfidf_embeddings_10_features_os.pkl')
# joblib.dump(clf, model_path)

['models\\random_forest_tfidf_embeddings_10_features_os.pkl']

In [44]:
# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.93      0.89      4229
           1       0.72      0.53      0.61      1397

    accuracy                           0.83      5626
   macro avg       0.79      0.73      0.75      5626
weighted avg       0.82      0.83      0.82      5626



In [239]:
best_params = {'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': False}
# best_params = {'n_estimators': 400,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 'sqrt',
#  'max_depth': None,
#  'bootstrap': False}

In [240]:
# train RFC with balanced_data - under sampling
clf = RandomForestClassifier(**best_params)
clf.fit(X_train_u_balanced,y_train_u_balanced)

RandomForestClassifier(bootstrap=False, max_depth=90, min_samples_split=5,
                       n_estimators=800)

In [251]:
# save random forest model
save_folder = 'models'
os.makedirs(save_folder, exist_ok = True)
model_path = os.path.join(save_folder, 'rfc_f1_59_4300_356.pkl')
joblib.dump(clf, model_path)

['models\\rfc_f1_59_4300_356.pkl']

In [46]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.78      0.85      4229
           1       0.55      0.80      0.65      1397

    accuracy                           0.79      5626
   macro avg       0.74      0.79      0.75      5626
weighted avg       0.83      0.79      0.80      5626



# test with gold data RFC US and OS without hyperparameter tuned rfc

In [242]:
# load test dataset 
test_df = pd.read_csv('created_new_test_data/english_test_11_2.csv') # without label
# same test data but with actual label
gold_df = pd.read_csv('data/CT24_checkworthy_english/CT24_checkworthy_english/test_english_gold/CT24_checkworthy_english_test_gold.tsv',delimiter = '\t')

In [55]:
# test_sf.sample(2)

In [243]:
text = test_df['text-tfidf']
# vectorizer_path = 'models/tfidf_vectorizer_4300.pkl'
# tfidf_vect = joblib.load(vectorizer_path) ## loading the trained tfidf vectorizer
tfidf_features = tfidf_vect.transform(text)

In [244]:
additional_features = test_df[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

In [245]:
test_df['converted_embedding'] = test_df['embedding'].apply(convert_to_numpy_array)

In [179]:
test_df['converted_embedding'].iloc[0].shape

(768,)

In [246]:
# pca = PCA(n_components = 356)
reduced_embeddings = pca.transform(test_df['converted_embedding'].to_list()) # the pcs from training data

In [247]:
embeddings_np = np.array(reduced_embeddings)
tfidf_features_np = tfidf_features.toarray()
additional_features_np = np.array(additional_features)
print(embeddings_np.shape)

(341, 356)


In [248]:
X_test = np.hstack([tfidf_features_np,embeddings_np, additional_features_np])

In [249]:
# load model rfc us
# model_path_rfc_us = 'models/random_forest_tfidf_embeddings_10_features_us.pkl'
# model_rfc_us = joblib.load(model_path_rfc_us)
# predictions_rfc_us = model_rfc_us.predict(X_test)
# predictions_rfc_us

predictions_rfc_us = clf.predict(X_test)
# # # load model rfc os - MEMORY ERROR
# model_path_rfc_os = 'models/random_forest_tfidf_embeddings_10_features_os.pkl'
# model_rfc_os = joblib.load(model_path_rfc_os)
# predictions_rfc_os = model_rfc_os.predict(X)
# # predictions_rfc_os

In [250]:
test_data_rfc_us = pd.DataFrame({
    'id': test_df['Sentence_id'],
    'predictions': predictions_rfc_us
})
# test_data_rfc_os = pd.DataFrame({
#     'id': test_df['Sentence_id'],
#     'predictions': predictions_rfc_os
# })
test_data_rfc_us['predictions'] = test_data_rfc_us['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])
# test_data_rfc_os['predictions'] = test_data_rfc_os['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])
## for LR
df_merged_rfc_us = pd.merge(gold_df, test_data_rfc_us, left_on='Sentence_id', right_on='id', how='inner')
# df_merged_rfc_os = pd.merge(gold_df, test_data_rfc_os, left_on='Sentence_id', right_on='id', how='inner')


gold_labels_us = df_merged_rfc_us['class_label'].tolist()
pred_labels_us = df_merged_rfc_us['predictions'].tolist()

# gold_labels_os = df_merged_rfc_os['class_label'].tolist()
# pred_labels_os = df_merged_rfc_os['predictions'].tolist()

print(f"RFC US")
acc = accuracy_score(gold_labels_us, pred_labels_us)
precision = precision_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')
recall = recall_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')
f1 = f1_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')

# Print the metrics
print(f"Accuracy: {acc:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# print(f"RFC OS")
# acc = accuracy_score(gold_labels_os, pred_labels_os)
# precision = precision_score(gold_labels_os, pred_labels_os, pos_label='Yes', average='binary')
# recall = recall_score(gold_labels_os, pred_labels_os, pos_label='Yes', average='binary')
# f1 = f1_score(gold_labels_os, pred_labels_os, pos_label='Yes', average='binary')

# # Print the metrics
# print(f"Accuracy: {acc}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

RFC US
Accuracy: 0.76
Precision: 0.53
Recall: 0.67
F1 Score: 0.59


### NOW  CREAETE A NEW MODEL RFC  WITH TUNED HYPERPARAMETER FROM DEV_TRAIN data,  train on UNDER SAMPLED DATA, test on gold data

In [59]:
best_params = {'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': False}

In [60]:
clf = RandomForestClassifier(**best_params)
clf.fit(X_train_u_balanced,y_train_u_balanced)

RandomForestClassifier(bootstrap=False, max_depth=90, min_samples_split=5,
                       n_estimators=800)

In [62]:
predictions_rfc_us_hpt = clf.predict(X) 

In [64]:
test_data_rfc_us_hpt = pd.DataFrame({
    'id': test_df['Sentence_id'],
    'predictions': predictions_rfc_us_hpt
})
# test_data_rfc_os = pd.DataFrame({
#     'id': test_df['Sentence_id'],
#     'predictions': predictions_rfc_os
# })
test_data_rfc_us_hpt['predictions'] = test_data_rfc_us_hpt['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])
# test_data_rfc_os['predictions'] = test_data_rfc_os['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])
## for LR
df_merged_rfc_us_hpt = pd.merge(gold_df, test_data_rfc_us_hpt, left_on='Sentence_id', right_on='id', how='inner')
# df_merged_rfc_os = pd.merge(gold_df, test_data_rfc_os, left_on='Sentence_id', right_on='id', how='inner')


gold_labels_us = df_merged_rfc_us_hpt['class_label'].tolist()
pred_labels_us = df_merged_rfc_us_hpt['predictions'].tolist()

# gold_labels_os = df_merged_rfc_os['class_label'].tolist()
# pred_labels_os = df_merged_rfc_os['predictions'].tolist()

print(f"RFC US")
acc = accuracy_score(gold_labels_us, pred_labels_us)
precision = precision_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')
recall = recall_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')
f1 = f1_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')

# Print the metrics
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


RFC US
Accuracy: 0.7683284457478006
Precision: 0.5463917525773195
Recall: 0.6022727272727273
F1 Score: 0.572972972972973


In [61]:
# save random forest model
save_folder = 'models'
os.makedirs(save_folder, exist_ok = True)
model_path = os.path.join(save_folder, 'random_forest_tfidf_embeddings_10_features_us_best_params.pkl')
joblib.dump(clf, model_path)

['models\\random_forest_tfidf_embeddings_10_features_us_best_params.pkl']

In [65]:
# train LR with balanced_data - over sampling
# lr = LogisticRegression(max_iter=1000)
# lr.fit(X_train_balanced,y_train_balanced)

In [66]:
# save linear regression model
# save_folder = 'models'
# os.makedirs(save_folder, exist_ok = True)
# model_path = os.path.join(save_folder, 'linear_regression_tfidf_embeddings_10_features_os.pkl')
# joblib.dump(lr, model_path)

In [67]:
# y_pred = lr.predict(X_test)
# print(classification_report(y_test, y_pred))

In [68]:
# train LR with balanced_data - under sampling
# lr = LogisticRegression(max_iter=1000)
# lr.fit(X_train_u_balanced,y_train_u_balanced)

In [69]:
# save linear regression model
# save_folder = 'models'
# os.makedirs(save_folder, exist_ok = True)
# model_path = os.path.join(save_folder, 'linear_regression_tfidf_embeddings_10_features_us.pkl')
# joblib.dump(lr, model_path)

In [70]:
# y_pred = lr.predict(X_test)
# print(classification_report(y_test, y_pred))

## Test with gold dataset for Liner regression over sampling and under sampling

In [71]:
# load model LR us
# model_path_lr_us = 'models/linear_regression_tfidf_embeddings_10_features_us.pkl'
# model_lr_us = joblib.load(model_path_lr_us)
# predictions_lr_us = model_lr_us.predict(X)
# # predictions_rfc_us

# # # load model LR os
# model_path_lr_os = 'models/linear_regression_tfidf_embeddings_10_features_os.pkl'
# model_lr_os = joblib.load(model_path_lr_os)
# predictions_lr_os = model_lr_os.predict(X)
# # predictions_rfc_os

In [72]:
# test_data_lr_us = pd.DataFrame({
#     'id': test_df['Sentence_id'],
#     'predictions': predictions_lr_us
# })
# test_data_lr_os = pd.DataFrame({
#     'id': test_df['Sentence_id'],
#     'predictions': predictions_lr_os
# })
# test_data_lr_us['predictions'] = test_data_lr_us['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])
# test_data_lr_os['predictions'] = test_data_lr_os['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])
# ## for LR
# df_merged_lr_us = pd.merge(gold_df, test_data_lr_us, left_on='Sentence_id', right_on='id', how='inner')
# df_merged_lr_os = pd.merge(gold_df, test_data_lr_os, left_on='Sentence_id', right_on='id', how='inner')


# gold_labels_us = df_merged_lr_us['class_label'].tolist()
# pred_labels_us = df_merged_lr_us['predictions'].tolist()

# gold_labels_os = df_merged_lr_os['class_label'].tolist()
# pred_labels_os = df_merged_lr_os['predictions'].tolist()

# print(f"LR US")
# acc = accuracy_score(gold_labels_us, pred_labels_us)
# precision = precision_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')
# recall = recall_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')
# f1 = f1_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')

# # Print the metrics
# print(f"Accuracy: {acc}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

# print(f"LR OS")
# acc = accuracy_score(gold_labels_os, pred_labels_os)
# precision = precision_score(gold_labels_os, pred_labels_os, pos_label='Yes', average='binary')
# recall = recall_score(gold_labels_os, pred_labels_os, pos_label='Yes', average='binary')
# f1 = f1_score(gold_labels_os, pred_labels_os, pos_label='Yes', average='binary')

# # Print the metrics
# print(f"Accuracy: {acc}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

## Using other models - hyperparameter tuning - train - test

## validation data process start

In [19]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\looka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\looka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
dev_train.columns

Index(['Sentence_id', 'Text', 'class_label', 'Tokens', 'text_length',
       'cleaned_text', 'cleaned_text_length', 'roberta_sentiment',
       'roberta_sent_pos', 'roberta_sent_neg', 'roberta_sent_mixed',
       'roberta_sentiment_class', 'count_tokens', 'counts', 'names',
       'organizations', 'dates', 'punctuation_count', 'embedding',
       'converted_embedding', 'flattened_embedding', 'label'],
      dtype='object')

In [41]:
punct_list = list(string.punctuation)
def joined_tokens(row):
    tokens = row['Tokens']
    token_list = ast.literal_eval(tokens)
    if len(token_list)==0:
        text = ''.join([char for char in row['Text'] if char not in string.punctuation]) #
        text_tokens = text.lower().split()
    else:
        text_tokens = token_list
        
    filtered_tokens = [word for word in text_tokens if word.lower() not in punct_list]
    
    return ' '.join(filtered_tokens)


# def joined_tokens(row):
#     tokens = row['Tokens']
#     print(tokens)
#     token_list = ast.literal_eval(tokens)
#     print(token_list)
#     if len(token_list) == 0:
#         text_tokens = row['Text'].lower().split()  # Tokenize the text
#         print(text_tokens)
#     else:
#         text_tokens = token_list
    
#     # Remove stop words
#     # filtered_tokens = [word for word in text_tokens if word.lower() not in stop_words]
    
#     return ' '.join(text_tokens)

In [10]:
nlp = spacy.load('en_core_web_sm')
def lemmatiz_text(text):
    doc = nlp(text)
    lemma_tokens = [token.lemma_ for token in doc]
    lemma_text = ' '.join(lemma_tokens)
    return lemma_text

In [42]:
dev_train['joined_tokens'] = dev_train.apply(joined_tokens, axis=1)
# dev_train['text-tfidf'] = dev_train['joined_tokens'].apply(lemmatiz_text)

In [43]:
# dev_train[dev_train['joined_tokens'].isnull() == True]
dev_train['joined_tokens'].iloc[736]

'we do not have to do it'

In [12]:
text = dev_train['text-tfidf']
vectorizer_path = 'models/tfidf_vectorizer_6300.pkl'
tfidf_vect = joblib.load(vectorizer_path)
tfidf_features = tfidf_vect.transform(text)

In [13]:
tfidf_features.shape

(1032, 6300)

In [17]:
dev_train.to_csv('created_new_dev_data/second_dev_train.csv')

In [16]:
dev_train.sample()

,Sentence_id,Text,class_label,Tokens,text_length,cleaned_text,cleaned_text_length,roberta_sentiment,roberta_sent_pos,roberta_sent_neg,...,names,organizations,dates,punctuation_count,embedding,converted_embedding,flattened_embedding,label,joined_tokens,text-tfidf
493,10465,"Now, I have a plan to fix this problem and it ...",No,"['plan', 'fix', 'problem', 'got', 'energy', 'i...",85,plan fix problem got energy independence,40,"{'positive_score': 0.31958216428756714, 'negat...",0.319582,0.067757,...,0,0,0,0,[[ 1.15189850e-01 2.02788115e-01 -1.70146152e...,[[ 1.15189850e-01 2.02788115e-01 -1.70146152e...,[ 1.15189850e-01 2.02788115e-01 -1.70146152e-...,0,plan fix problem got energy independence,plan fix problem get energy independence


In [78]:
def convert_to_numpy_array(embd_str):
    #print(f"string: {embd_str}")
    #print(type(embd_str))
    embd_str_clean = embd_str.replace("\n", " ").replace(" ", " ")
    #print(type(embd_str_clean))
    #print(f"string cleansing: {embd_str_clean}")
    embd_str_clean = embd_str_clean.strip('[]')
    #print(type(embd_str_clean))
    #print(f"removing the outer brackets {embd_str_clean}")
    embd_list = embd_str_clean.split()
    #print(f"Split the string into individual values {embd_list}")
    embed_array = np.array(embd_list, dtype = float)
    #print(f"list to array {embed_array}")
    return embed_array

In [80]:
additional_features = dev_train[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

In [81]:
dev_train['converted_embedding'] = dev_train['embedding'].apply(convert_to_numpy_array)

In [83]:
pca = PCA(n_components = 50)
reduced_embeddings = pca.fit_transform(dev_train['converted_embedding'].to_list())

In [84]:
embeddings_np = np.array(reduced_embeddings)
tfidf_features_np = tfidf_features.toarray()
additional_features_np = np.array(additional_features)

In [85]:
X_eval = np.hstack([tfidf_features_np,embeddings_np, additional_features_np])
y_eval = dev_train['label']

### validation data process end

## XGBoost

In [65]:
# {'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 
#  'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 
#  'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 
#  'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 
#  'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': None, 
#  'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 
#  'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}
best_params = {
 'learning_rate': 0.09460545700275623,
 'max_depth': 10, #20
 'n_estimators': 400,
 'subsample': 0.8, # 0.5115069174630871
    'colsample_bytree': 0.8,
    'tree_method': 'hist'
}

In [109]:
# X , y - real data
# X_train_u_balanced, y_train_u_balanced - under sampleed data
# test_df - without label
# gold_df - with label

In [2]:
# # new_df['class_label'].value_counts()
# scale_pos_weight = 17088 / 5413
# # print(scale_pos_weight)
# xgb_model = XGBClassifier(**best_params, random_state = 42, scale_pos_weight = scale_pos_weight, early_stopping_rounds = 10, eval_metric = 'logloss')
# xgb_model.fit(X_train, y_train, eval_set=[(X_eval, y_eval)], verbose = True)

In [76]:
# xgb = XGBClassifier()
# print(xgb.get_params())

{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


In [131]:
y_pred_xgb_imblnc = xgb_model.predict(X_test)
# print(classification_report(y_test, y_pred))

In [132]:
test_data_xgb = pd.DataFrame({
    'id': test_df['Sentence_id'],
    'predictions': y_pred_xgb_imblnc
})
# test_data_rfc_os = pd.DataFrame({
#     'id': test_df['Sentence_id'],
#     'predictions': predictions_rfc_os
# })
test_data_xgb['predictions'] = test_data_xgb['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])
# test_data_rfc_os['predictions'] = test_data_rfc_os['predictions'].apply([lambda x: 'No' if x == 0 else 'Yes'])
## for LR
test_data_merged_xgb = pd.merge(gold_df, test_data_xgb, left_on='Sentence_id', right_on='id', how='inner')
# df_merged_rfc_os = pd.merge(gold_df, test_data_rfc_os, left_on='Sentence_id', right_on='id', how='inner')


gold_labels_us = test_data_merged_xgb['class_label'].tolist()
pred_labels_us = test_data_merged_xgb['predictions'].tolist()

# gold_labels_os = df_merged_rfc_os['class_label'].tolist()
# pred_labels_os = df_merged_rfc_os['predictions'].tolist()

print(f"XGB on original imbalance dataset")
acc = accuracy_score(gold_labels_us, pred_labels_us)
precision = precision_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')
recall = recall_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')
f1 = f1_score(gold_labels_us, pred_labels_us, pos_label='Yes', average='binary')

# Print the metrics
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


XGB on original imbalance dataset
Accuracy: 0.7302052785923754
Precision: 0.4803921568627451
Recall: 0.5568181818181818
F1 Score: 0.5157894736842106


## SVM

In [ ]:
svm = SVC(class_weight='balanced', random_state=42)
svm.fit(X_train, y_train)

In [ ]:
y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))

## KNN

In [ ]:
knn = KNeighborsClassifier(weights='distance')
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))

## GNB

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)

In [ ]:
y_pred = nb.predict(X_test)
print(classification_report(y_test, y_pred))

## AUGMENTATION TECHNIQUE